In [28]:
import pandas as pd
import sklearn.metrics

In [29]:
df = pd.read_csv("Data/Final_data.csv")
df.head()

,Unnamed: 0_x,ArrDel15,ArrDelayMinutes,ArrTime,CRSArrTime,CRSDepTime,DayofMonth,DepDel15,DepDelayMinutes,DepTime,...,pressure_y,cloudcover_y,DewPointF_y,WindGustKmph_y,tempF_y,WindChillF_y,humidity_y,time_y,date_y,airport_y
0,0,0.0,8.0,1610.0,1602,745,1,0.0,0.0,741.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK
1,557,1.0,25.0,1628.0,1603,1330,1,1.0,38.0,1408.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK
2,282,0.0,0.0,1540.0,1600,1310,1,0.0,0.0,1306.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK
3,51602,0.0,0.0,1508.0,1530,700,1,0.0,0.0,659.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK
4,1506,0.0,0.0,1600.0,1624,800,1,0.0,0.0,758.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK


In [30]:
df = df.drop(columns=['Unnamed: 0_x', 'time_x', 'date_x','time_y', 'date_y', 'FlightDate', 'Unnamed: 0_y', 'airport_y', 
                      'airport_x', 'ArrTime', 'Time_new_dept', 'Time_new_arr','Unnamed: 0'])
df.head()

,ArrDel15,ArrDelayMinutes,CRSArrTime,CRSDepTime,DayofMonth,DepDel15,DepDelayMinutes,DepTime,DestAirportID,Month,...,weatherCode_y,precipMM_y,visibility_y,pressure_y,cloudcover_y,DewPointF_y,WindGustKmph_y,tempF_y,WindChillF_y,humidity_y
0,0.0,8.0,1602,745,1,0.0,0.0,741.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74
1,1.0,25.0,1603,1330,1,1.0,38.0,1408.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74
2,0.0,0.0,1600,1310,1,0.0,0.0,1306.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74
3,0.0,0.0,1530,700,1,0.0,0.0,659.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74
4,0.0,0.0,1624,800,1,0.0,0.0,758.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74


In [31]:
df.columns

Index(['ArrDel15', 'ArrDelayMinutes', 'CRSArrTime', 'CRSDepTime', 'DayofMonth',
       'DepDel15', 'DepDelayMinutes', 'DepTime', 'DestAirportID', 'Month',
       'Origin', 'OriginAirportID', 'Quarter', 'Year', 'Dest',
       'windspeedKmph_x', 'winddirDegree_x', 'weatherCode_x', 'precipMM_x',
       'visibility_x', 'pressure_x', 'cloudcover_x', 'DewPointF_x',
       'WindGustKmph_x', 'tempF_x', 'WindChillF_x', 'humidity_x',
       'windspeedKmph_y', 'winddirDegree_y', 'weatherCode_y', 'precipMM_y',
       'visibility_y', 'pressure_y', 'cloudcover_y', 'DewPointF_y',
       'WindGustKmph_y', 'tempF_y', 'WindChillF_y', 'humidity_y'],
      dtype='object')

In [32]:
from sklearn import preprocessing
label = preprocessing.LabelEncoder()
df['Origin'] = label.fit_transform(df['Origin'])
df['Dest'] = label.fit_transform(df['Dest'])

In [33]:
y = df[['ArrDel15', 'ArrDelayMinutes']]
X = df.drop(columns = ['ArrDel15', 'ArrDelayMinutes'])

In [34]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.70, random_state = 50)

In [35]:
y_train_classification, y_test_classification = y_train["ArrDel15"], y_test["ArrDel15"]
y_test_regression = y_test["ArrDelayMinutes"]

In [36]:
import xgboost
from xgboost import XGBClassifier
xgboost_model = XGBClassifier()
xbgoost_classifier = xgboost_model.fit(X_train, y_train_classification)

In [37]:
y_pred_xgboost = xgboost_model.predict(X_test)

In [38]:
sklearn.metrics.classification_report(y_test_classification, y_pred_xgboost)

'              precision    recall  f1-score   support\n\n         0.0       0.92      0.98      0.95    438809\n         1.0       0.90      0.70      0.79    116418\n\n    accuracy                           0.92    555227\n   macro avg       0.91      0.84      0.87    555227\nweighted avg       0.92      0.92      0.92    555227\n'

In [39]:
X_test["Predicted_ArrDel15"] = y_pred_xgboost
X_test = X_test.loc[:][X_test["Predicted_ArrDel15"]==1]
X_test = X_test.drop(["Predicted_ArrDel15"],axis=1)
X_test.shape

<ipython-input-39-b8ccfceb4856>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["Predicted_ArrDel15"] = y_pred_xgboost


(90636, 37)

In [40]:
y_test_regression_new= pd.DataFrame(y_test_regression)

y_test_regression_new["Predicted_ArrDel15"] = y_pred_xgboost

y_test_regression_new = y_test_regression_new.loc[y_test_regression_new["Predicted_ArrDel15"] == 1]

y_test_regression_new = y_test_regression_new.drop(["Predicted_ArrDel15"], axis = 1)

In [41]:
y_test_regression_new = pd.Series(y_test_regression_new["ArrDelayMinutes"])

In [42]:
X_test.shape, y_test_regression_new.shape

((90636, 37), (90636,))

In [43]:
X_train["ArrDel15"] = y_train["ArrDel15"]
X_train = X_train.loc[X_train["ArrDel15"]==1]
X_train = X_train.drop("ArrDel15", axis = 1)
y_train_regression = y_train.loc[y_train["ArrDel15"] == 1]
y_train_regression = y_train_regression.drop("ArrDel15", axis = 1)

<ipython-input-43-cde8f05ca3c0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["ArrDel15"] = y_train["ArrDel15"]


In [44]:
X_train.shape, y_train_regression.shape, X_test.shape, y_test_regression_new.shape

((271516, 37), (271516, 1), (90636, 37), (90636,))

In [ ]:
from xgboost import XGBRegressor
xgboost_regressor = XGBRegressor()
xgboost_regressor.fit(X_train,y_train_regression)
y_pred_xgboost_regression = xgboost_regressor.predict(X_test)

In [ ]:
from sklearn import metrics
import numpy as np
mean_absolute_error=metrics.mean_absolute_error(y_test_regression_new, y_pred_xgboost_regression) 
MSE=metrics.mean_squared_error(y_test_regression_new, y_pred_xgboost_regression) 
RMSE= np.sqrt(MSE)
R2=metrics.r2_score(y_test_regression_new, y_pred_xgboost_regression)
print(mean_absolute_error)
print(MSE)
print(RMSE)
print(R2)